# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`.

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure.

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
pip install requests beautifulsoup4 pandas


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
url = "http://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)

print(response.status_code)  # Check the status code of the page
print(response.text[:500])   # Print the first 500 characters


200


<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8"


In [4]:
soup = BeautifulSoup(response.text, "html.parser")  # Parse HTML
books = soup.find_all("article", class_="product_pod")  # Find all books

print(len(books))  # Print how many books are availableır


20


In [5]:
first_book = books[0]  # Take the first book

# Pull book title
title = first_book.h3.a["title"]
print("Title:", title)

# Check the price
price = first_book.find("p", class_="price_color").text
print("Price:", price)

# Pull the score
rating_class = first_book.find("p", class_="star-rating")["class"]
rating_text = rating_class[1]  # There will be a list like ['star-rating', 'Four']
rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
rating = rating_map.get(rating_text, 0)
print("Rating:", rating)

# Pull stock status
availability = first_book.find("p", class_="instock availability").text.strip()
print("Availability:", availability)


Title: A Light in the Attic
Price: Â£51.77
Rating: 3
Availability: In stock


In [6]:
books_data = []

for book in books:
    title = book.h3.a["title"]
    # Extract the price, remove the pound sign and any non-breaking spaces, and convert to float
    price = float(book.find("p", class_="price_color").text[1:].replace("£", "").replace("Â", "").strip())
    rating_text = book.find("p", class_="star-rating")["class"][1]
    rating = rating_map.get(rating_text, 0)
    availability = book.find("p", class_="instock availability").text.strip()

    books_data.append([title, price, rating, availability])

df = pd.DataFrame(books_data, columns=["Title", "Price (£)", "Rating", "Availability"])
print(df.head())  # Print the first 5 books


                                   Title  Price (£)  Rating Availability
0                   A Light in the Attic      51.77       3     In stock
1                     Tipping the Velvet      53.74       1     In stock
2                             Soumission      50.10       1     In stock
3                          Sharp Objects      47.82       4     In stock
4  Sapiens: A Brief History of Humankind      54.23       5     In stock


In [16]:
books_data = []
base_url = "http://books.toscrape.com/catalogue/page-{}.html"

for page in range(1, 51):  # Navigate pages 1 to 50
    url = base_url.format(page)
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Page {page} failed to load!")
        continue

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    for book in books:
        title = book.h3.a["title"]
        price = float(book.find("p", class_="price_color").text[1:].replace("£", "").replace("Â", "").strip())
        rating_text = book.find("p", class_="star-rating")["class"][1]
        rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
        rating = rating_map.get(rating_text, 0)
        availability = book.find("p", class_="instock availability").text.strip()

        books_data.append([title, price, rating, availability])

    print(f"Page {page} completed.")  # Let's see which page is finished

df = pd.DataFrame(books_data, columns=["Title", "Price (£)", "Rating", "Availability"])
print(df.head())  # Print the first 5 books


Page 1 completed.
Page 2 completed.
Page 3 completed.
Page 4 completed.
Page 5 completed.
Page 6 completed.
Page 7 completed.
Page 8 completed.
Page 9 completed.
Page 10 completed.
Page 11 completed.
Page 12 completed.
Page 13 completed.
Page 14 completed.
Page 15 completed.
Page 16 completed.
Page 17 completed.
Page 18 completed.
Page 19 completed.
Page 20 completed.
Page 21 completed.
Page 22 completed.
Page 23 completed.
Page 24 completed.
Page 25 completed.
Page 26 completed.
Page 27 completed.
Page 28 completed.
Page 29 completed.
Page 30 completed.
Page 31 completed.
Page 32 completed.
Page 33 completed.
Page 34 completed.
Page 35 completed.
Page 36 completed.
Page 37 completed.
Page 38 completed.
Page 39 completed.
Page 40 completed.
Page 41 completed.
Page 42 completed.
Page 43 completed.
Page 44 completed.
Page 45 completed.
Page 46 completed.
Page 47 completed.
Page 48 completed.
Page 49 completed.
Page 50 completed.
                                   Title  Price (£)  Rating

In [8]:
# Home page URL
base_url = "http://books.toscrape.com/catalogue/page-1.html"
site_url = "http://books.toscrape.com/catalogue/"

# Pull the HTML code of the page
response = requests.get(base_url)
soup = BeautifulSoup(response.text, "html.parser")

# Find links to all books
books = soup.find_all("h3")

for book in books:
    link = book.find("a")["href"]  # Kitap detay linki (örnek: “catalogue/a-light-in-the-attic_1000/index.html”
    full_link = site_url + link  # Create full link
    print(full_link)  # Print the full link to the book detail page


http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
http://books.toscrape.com/catalogue/soumission_998/index.html
http://books.toscrape.com/catalogue/sharp-objects_997/index.html
http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
http://books.toscrape.com/catalogue/the-requiem-red_995/index.html
http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html
http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html
http://books.toscrape.com/catalogue/the-black-maria_991/index.html
http://books.toscrape.com/catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html
http://books.toscrape.com/cata

In [9]:
for book in books:
    link = book.find("a")["href"]
    full_link = site_url + link

    # Print full link to book detail page
    book_response = requests.get(full_link)
    book_soup = BeautifulSoup(book_response.text, "html.parser")

    # Find the description
    description_tag = book_soup.find("meta", {"name": "description"})

    if description_tag:
        description = description_tag["content"].strip()
    else:
        description = "No explanation"

    print(f"Book: {book.find('a').text} \nExplanation: {description}\n")


Book: A Light in the ... 
Explanation: It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone down here'sGot

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
site_url = "http://books.toscrape.com/catalogue/"
base_url = "http://books.toscrape.com/catalogue/page-{}.html"

# List to store the results
books_data = []

# Page number (there are 50 pages)
for page in range(1, 51):
    url = base_url.format(page)
    response = requests.get(url)

    # Check if the page is valid
    if response.status_code != 200:
        break  # Exit the loop if there is no page

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("h3")

    for book in books:
        # Find book detail link and generate full URL
        link = book.find("a")["href"]
        full_link = site_url + link

        # Go to book detail page
        book_response = requests.get(full_link)
        book_soup = BeautifulSoup(book_response.text, "html.parser")

        # Pull book information
        title = book.find("a").text.strip()
        price = float(book_soup.find("p", class_="price_color").text.strip().replace("£", "").replace("Â", "").replace(" ", ""))
        rating = book_soup.find("p", class_="star-rating")["class"][1]
        availability = book_soup.find("p", class_="instock availability").text.strip()
        genre = book_soup.find("ul", class_="breadcrumb").find_all("li")[2].text.strip()

        # Pull description
        description_tag = book_soup.find("meta", {"name": "description"})
        description = description_tag["content"].strip() if description_tag else "No Description"

        # UPC code extraction
        upc = book_soup.find("th", text="UPC").find_next("td").text.strip()

        # ⭐⭐⭐ HERE WE ADD A FILTER ⭐⭐⭐
        if (rating in ["Four", "Five"]) and (price <= 20):
            # Add data to list
            books_data.append([upc, title, price, rating, genre, availability, description])

# Create DataFrame
df = pd.DataFrame(books_data, columns=["UPC", "Title", "Price (£)", "Rating", "Genre", "Availability", "Description"])

# Save as CSV
#df.to_csv("books_data.csv", index=False)

print("All pages processed and data saved in books_data.csv file!")


<ipython-input-10-bc6eeaa685d4>:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = book_soup.find("th", text="UPC").find_next("td").text.strip()


All pages processed and data saved in books_data.csv file!


In [12]:
df.head()

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,Five,Young Adult,In stock (19 available),Aaron Ledbetterâs future had been planned ou...
1,6258a1f6a6dcfe50,The Four Agreements: A ...,17.66,Five,Spirituality,In stock (18 available),"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,Five,Philosophy,In stock (18 available),A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems ...,14.27,Four,Poetry,In stock (16 available),"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,Four,Sequential Art,In stock (16 available),"Every summer, Rose goes with her mom and dad t..."


In [13]:
df['Rating'].value_counts()

,count
Rating,
Five,42
Four,33


In [15]:
df['Price (£)'].max()

19.69